In [1]:
### import library

import pandas as pd

In [2]:
### Load the patient dataset and show summary with info().

In [4]:
# Load datasets

patient_df = pd.read_csv('Patient_Data.csv')
billing_df = pd.read_csv('Billing_Data.csv')

In [5]:
patient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


In [6]:
billing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 252.0 bytes


In [10]:
### summary

patient_df.describe()

,PatientID,BillAmount,ReceptionistID
count,6.000000,4.000000,6.000000
mean,102.666667,5925.000000,1.666667
std,1.632993,1192.686044,0.816497
min,101.000000,5000.000000,1.000000
25%,101.250000,5000.000000,1.000000
50%,102.500000,5600.000000,1.500000
75%,103.750000,6525.000000,2.000000
max,105.000000,7500.000000,3.000000


In [9]:
billing_df.describe()

,PatientID,InsuranceCovered,FinalAmount
count,5.000000,5.000000,5.000000
mean,103.000000,2000.000000,3740.000000
std,1.581139,790.569415,798.749022
min,101.000000,1000.000000,3000.000000
25%,102.000000,1500.000000,3200.000000
50%,103.000000,2000.000000,3500.000000
75%,104.000000,2500.000000,4000.000000
max,105.000000,3000.000000,5000.000000


In [12]:
# Remove duplicate patient records based on PatientID

patient_df_cleaned = patient_df.drop_duplicates(subset=['PatientID']).copy()

In [13]:
# Drop administrative columns

patient_df_cleaned = patient_df_cleaned.drop(columns=['ReceptionistID', 'CheckInTime'])

In [14]:
# Fill missing BillAmount values with the mean bill amount
mean_bill = patient_df_cleaned['BillAmount'].mean()
patient_df_cleaned['BillAmount'] = patient_df_cleaned['BillAmount'].fillna(mean_bill)

In [15]:
# Select relevant columns for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount']

billing_relevant_df = patient_df_cleaned[['PatientID', 'Department', 'Doctor', 'BillAmount']]

In [16]:
# 2. Groupby to find total bill amount per department

dept_total_bill = billing_relevant_df.groupby('Department')['BillAmount'].sum().reset_index()

In [17]:
# 3. Merge billing dataset (Billing_Data.csv) with patient dataset on PatientID

merged_df = pd.merge(billing_relevant_df, billing_df[['PatientID']], on='PatientID', how='left')

In [18]:
# 4. Concatenate an additional DataFrame that contains new patients for the current week (row-wise)

new_patients_data = {
    'PatientID': [106, 107],
    'Department': ['Orthopedics', 'Neurology'],
    'Doctor': ['Dr. Lee', 'Dr. John'],
    'BillAmount': [4500.0, 5200.0]
}

In [19]:
new_patients_df = pd.DataFrame(new_patients_data)
concatenated_rows_df = pd.concat([billing_relevant_df, new_patients_df], ignore_index=True)

In [21]:
### Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise

insurance_data = pd.merge(concatenated_rows_df[['PatientID']], billing_df, on='PatientID', how='left')[['InsuranceCovered', 'FinalAmount']]
final_df = pd.concat([concatenated_rows_df, insurance_data], axis=1)

In [22]:
# Display results
print("Total Bill Amount per Department:")
print(dept_total_bill)
print("\nFinal Cleaned and Merged Dataset:")
print(final_df)

Total Bill Amount per Department:
    Department    BillAmount
0   Cardiology  11200.000000
1  Dermatology   6233.333333
2    Neurology   6233.333333
3  Orthopedics   7500.000000

Final Cleaned and Merged Dataset:
   PatientID   Department     Doctor   BillAmount  InsuranceCovered  \
0        101   Cardiology  Dr. Smith  5000.000000            2000.0   
1        102    Neurology   Dr. John  6233.333333            1500.0   
2        103  Orthopedics    Dr. Lee  7500.000000            2500.0   
3        104   Cardiology  Dr. Smith  6200.000000            3000.0   
4        105  Dermatology   Dr. Rose  6233.333333            1000.0   
5        106  Orthopedics    Dr. Lee  4500.000000               NaN   
6        107    Neurology   Dr. John  5200.000000               NaN   

   FinalAmount  
0       3000.0  
1       3500.0  
2       5000.0  
3       3200.0  
4       4000.0  
5          NaN  
6          NaN  
